### Task 1.1


In [4]:
def extract_words_tokens(any_string: str):
    without_space_string = any_string.strip().split()
    num_words = len(without_space_string)
    num_tokens = len(any_string)
    return(print(any_string, ":", "num_words:", num_words, "and", "num_tokens:", num_tokens, "respectively"))

Hello, world! This is a test. : num_words: 6 and num_tokens: 29 respectively
